In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time

os.chdir(r'C:\Users\Lane Capital\OneDrive\Desktop\repos\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol
from Strategies import Utils
# from MyFunctions.EMA import EMA, EMA_1, MACD

sim = build_active_IBridgePy_plus('DU2628075') # Build an active trader to get connected to Interactive Brokers 

Try brokerClient IBinsync port=7496
port=7496 is correct and set port to userConfig.brokerClientConfig.IB_CLIENT
Connected to IB is successful using port=7496
##    ACCOUNT Balance  DU2628075  ##
CASH=515843.85
portfolio_value=314785.84
positions_value=201213.77
##    POSITIONS DU2628075   ##
Symbol Amount Cost_basis
STK,,,SPY,USD -527 381.37905085
##    NO any order    ##


In [ ]:
# functions

# get price -done
# - calc trailing stop - done
# update trailing stop

# confirm/update these
# - exit long 
# - exit short



# Do these second
# calculate exit price - long
# calculate exit price - Short

# Do these third 

# order short
# order long
# sell order short
# sell order long


# Later on
# - double enter
# - double exit
# double order short/long
# double order sell short/long
# - csv reader
# - csv writer

In [ ]:
def calcTrailingStop(context, price, mag: float = 5, percentBool: bool=True, shortBool: bool=False):
    """ This function takes inputs of:
    - Context structure
    - Price (USD)
    - Magnitude (USD or %), If percentage is used, use the magnitude of the percentage (i.e. 5, not 0.05 for 5%)
    - boolean flag which determines whether to use percentage or absolute value
    - Boolean flag for long or short sale
     and calculates a trailing stop price. """

    if shortBool: # We are holding short
        if percentBool: # Calculate trailing stop using percentage
            tsPriceNew=price.mid*(1+mag/100)
        else: # Calculate trailing stop using absolute value in USD
            tsPriceNew=price.mid+mag
    else: # We are holding long
        if percentBool: # Calculate trailing stop using percentage
            tsPriceNew=price.mid*(1-mag/100)
        else: # Calculate trailing stop using absolute value in USD
            tsPriceNew=price.mid-mag
            # TODO put context.tsPrice
    return tsPriceNew

def updateTrailingStop(context, price, mag: float = 5, tsPrice: float= np.NAN, percentBool: bool=True, shortBool: bool=False, newTsBool: bool=False):
    """ This function takes inputs of:
    - Context structure
    - Price (USD)
    - Magnitude (USD or %), If percentage is used, use the magnitude of the percentage (i.e. 5, not 0.05 for 5%)
    - Previous trailing stop price (if it exists), else leave empty
    - boolean flag which determines whether to use percentage or absolute value
    - Boolean flag for long or short sale
    - Boolean for whether or not this is the first time we are calculating the trailing stop price
     and determines whether or not to update the trailing price. 
     
     Conditions for updating trailing stop price:
     
     - We hold a short position and the price falls below  the previous trailing stop price
     - we hold a long position and the price rise above the previous trailing stop price
     
     # TODO confirm this is correct, we may have to adjust when the price goes above a previous peak, though
     this can be an easier way to calculate so we dont have to store previous maximum data

     """    
    
    if newTsBool:
        tsPriceNew=calcTrailingStop(context, price, mag=mag, percentBool=percentBool, shortBool=shortBool)
    elif shortBool and percentBool:
        if ts_price < price.mid*(1+mag/100):
            tsPriceNew=calcTrailingStop(context, price, mag=mag, percentBool=percentBool, shortBool=shortBool)
    elif shortBool and not percentBool:
        if ts_price < price.mid+mag:
            tsPriceNew=calcTrailingStop(context, price, mag=mag, percentBool=percentBool, shortBool=shortBool)
    elif not shortBool and percentBool:
        if ts_price > price.mid*(1+mag/100):
            tsPriceNew=calcTrailingStop(context, price, mag=mag, percentBool=percentBool, shortBool=shortBool)
    elif not shortBool and not percentBool:
        if ts_price > price.mid+mag:
            tsPriceNew=calcTrailingStop(context, price, mag=mag, percentBool=percentBool, shortBool=shortBool)
    else:
        tsPriceNew=tsPrice
        
    return tsPriceNew

In [ ]:
def getPrice(context):
    """ This function takes in a context structure and outputs a data frame with colmns 
    of last price, ask price, bid price, and mid price. Note when moving this function to the actual algo,
    update security to context.security and get rid of sim.show..."""
    
    # get all relevant prices
    priceAsk=sim.show_real_time_price(security,'ask_price')
    priceBid=sim.show_real_time_price(security,'bid_price')
    priceLast=sim.show_real_time_price(security,'last_price')
    priceMid=round((priceAsk + priceBid) / 2, 2) # calculate midpoint price
    
    # convert prices to a dataframe
    price=pd.DataFrame([np.array([priceLast, priceAsk, priceBid, priceMid])],
                       columns=['last', "ask", "bid", "mid"])
    return price

In [ ]:
sim.request_historical_data(symbol('SPY'), '1 min', '86400 S')


In [ ]:
security = symbol('SPY')
context=initializeJupyter()
price=getPrice(context)
tsPrice=calcTrailingStop(context, price, 5, percentBool=True, shortBool=True)

print(price.mid.values[0],tsPrice)

In [ ]:
# def testTrailingStopCalcs():

In [ ]:
# algo Notes

# calculate your ema's
# check if you should enter
#     if you own:
#         calculate your trailing stop
# check if you should exit
#     if exit flag:
#         place order for sale or purchase




In [ ]:
placing orders:

# 1. actually place the order:
    context.exit_order = order(context.security, exit_shares, LimitOrder(price_mid),
                                               accountCode='default')
# 2. check if order is filled:
#     2.a if the order is filled: reset everything:
        if get_order_status(context.exit_order) in [OrderStatus.FILLED]:
            context.exit_order = None
            flag_reset(context)
#     2.b if its not placed: modify order after N seconds:
            elif get_order_status(context.exit_order) not in [OrderStatus.FILLED]:
                i = int((sTime - context.exit_time).total_seconds())
                if i == 5:
                    print(get_order_status(context.exit_order))
                    modify_order(context.exit_order, newLimitPrice=price_mid)
                    context.exit_time = sTime

In [ ]:
def generateOrders(context,price):
    """ This function places the orders. Details to follow"""
    if context.exit_order is None: # Order has not been placed yet
        exitShares = -context.shares # number of shares to order
        context.exit_order = order(context.security, exit_shares, LimitOrder(price.mid), accountCode='default')
        context.exit_time = sTime # Save order time for future reference
    else: # Order has been placed
        if checkFillstatus(context): # Order has been filled
            context.exit_order = None # reset flags
            flag_reset(context)
        else: # Order has not been filled
            modifyOrder(context, price,sTime) # modify order to determine whether or not to lower the price

In [ ]:
future work: 
    smart pricing if mid price doesnt take
    smart order numbers

In [ ]:
def checkFillstatus(context):
    """This function checks the order status and returns a True if its filled,
    otherwise it returns a False"""
    return get_order_status(context.exit_order) in [OrderStatus.FILLED]

In [ ]:
def modifyOrder(context, price, sTime, N: int = 5):
    i = int((sTime - context.exit_time).total_seconds())
    if i>=N:
        newPrice=calculateLowerPrice(price, i)
        modify_order(context.exit_order, newLimitPrice=newPrice)
        context.exit_time = sTime

In [ ]:
def calculateLowerPrice(price, i):
    """This function takes in a price, an order, and number of time steps the 
    order has been tried to be filled and outputs a new price. A multipler is used as
    a function of a lookup table to calculate the new price."""
    
    # TODO find optimal lookup table
    
    multiplier = np.array([[0,5, 7, 9, 11, 50],
                      [0.5,0.5, 0.49, 0.48, 0.47, 0.47]]) # add a multiplier to manipulate price later
    mult = np.interp(i,multiplier[0],multiplier[1]) # interpolate i over multiplier
    newPrice = priceMid=(price.ask + price.bid)*mult # calculate new price based on lookup table of i
    
    return newPrice

In [ ]:
multiplier = np.zeros((2,25))
multiplier[0,:-1] = np.arange(6,6+2*(multiplier.shape[1]-1),2)
multiplier[0,-1] = 1000
multiplier[1,:] = np.ones((1,multiplier.shape[1]))*0.5

In [ ]:
multiplier = np.array([[0,5, 7, 9, 11, 50],
                      [0.5, 0.5, 0.49, 0.48, 0.47, 0.47]]) # add a multiplier to manipulate price later

mult = np.interp(6,multiplier[0],multiplier[1])

In [ ]:
price = pd.DataFrame([np.array((100,102.3,99.3,(102.3+99.3)/2))],columns=['last', "ask", "bid", "mid"])
for i in range(13):
    print(calculateLowerPrice(price,i)[0])

In [ ]:
31300/60/60

In [ ]:
sim.

In [2]:
np.round(3.45333,2)

3.45

In [ ]:
def initializeJupyter():
    initialData = [False, symbol('SPY'), False, False, False, 
        sim.request_historical_data(symbol('SPY'), '1 min', '16000 S'),
        sim.request_historical_data(symbol('SPY'), '5 mins', '31300 S'),
        [], None, False, None, None, None, None, 0.1, 0.2, False, False,
        False, False, None, None, None, None, None, None,
        sim.show_real_time_price(symbol('SPY'), 'last_price'),
        sim.show_real_time_price(symbol('SPY'), 'last_price'),100000.00]


    cols=["run_once", 
        "security", 
        "entry_flag", 
        "long_flag", 
        "short_flag", 
        "hist_1min",
        "hist_5min",
        "macd_list", 
        "entry_price", 
        "enter_flag", 
        "stop_price", 
        "new_stop_price",
        "ts_price_1",
        "ts_price_2",
        "trail_1", 
        "trail_2", 
        "new_ts_flag",
        "double_flag",
        "exit_flag",
        "TG_flag",
        "shares",
        "order",
        "double_order",
        "exit_order",
        "entry_time",
        "exit_time",
        "price_1", 
        "price_n", 
        "value", ]

    context=pd.DataFrame([np.array(initialData)],columns=cols)
    
    return context